In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import country_converter as coco

# load the world city dataframe for debugging purposes 
# worldcities = pd.read_csv('worldcities.csv', usecols = ['city', 'city_ascii', 'country', 'iso3'])

In [2]:
# plt.hist(survey_result_pub2018[survey_result_pub2018.ConvertedSalary != 'NA']['ConvertedSalary'].map(float))
# plt.show()

''' following shows using the converted salary seems to preserve most of more of the data '''
# print(survey_result_pub2018[survey_result_pub2018.Salary != 'NA'].Salary.size) # 50578
# print(survey_result_pub2018[(survey_result_pub2018.Currency != 'NA') & (survey_result_pub2018.Salary != 'NA')].Currency.size)  # 45150
# print(survey_result_pub2018[survey_result_pub2018.ConvertedSalary != 'NA'].ConvertedSalary.size) # 47702
# print(survey_result_pub2018.ConvertedSalary.size)    # 98855

' following shows using the converted salary seems to preserve most of more of the data '

In [3]:
# country may need to be cleaned as its user filled field
# columns with accessJobs and accessBenefits should change the title accordingly --> also lots of NA
# JobContactPriorities, JobEmailPriorities are in page 17 / 18

survey2018 = pd.read_csv('part_of_data/developer_survey_2018/survey_results_public.csv', 
                                    usecols = [
                                        'Hobby', 'Country', 'Student', 'YearsCoding', 'YearsCodingProf',
                                        'JobSatisfaction', 'CareerSatisfaction', 'Age', 'Gender', 
# add the following if the ranking for criteria to access a job
#                                         'DevType', 'AssessJob1', 'AssessJob2', 'AssessJob3', 'AssessJob4',
#                                         'AssessJob5', 'AssessJob6', 'AssessJob7', 'AssessJob8', 'AssessJob9', 'AssessJob10',
                                        'ConvertedSalary', 'LanguageWorkedWith', 'LanguageDesireNextYear', 
                                        'DatabaseWorkedWith', 'DatabaseDesireNextYear', 'PlatformWorkedWith', 
                                        'PlatformDesireNextYear', 'FrameworkWorkedWith', 'FrameworkDesireNextYear',
# interesting fields
                                        'AIFuture', 'EthicsChoice', 'EthicsResponsible', 'EthicalImplications',
                                        'StackOverflowRecommend', 'StackOverflowVisit', 'StackOverflowParticipate',
# interesting fields 2 
                                        'WakeTime', 'HoursComputer', 'HoursOutside', 'SkipMeals', 'Exercise', 
                                        'RaceEthnicity'
                                    ])
survey2018 = survey2018.dropna(thresh=6)
survey2018.reset_index(drop=True, inplace=True)

/Users/davidyeung/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (12,13,65,66,67,68,69,70,71,72,96,97,99,100,101,102,104,114,115,116,117,119,120,123,124) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
''' converting the country into isconverted_countrycountry converter '''
converted_country = pd.Series(coco.convert(names=survey2018.Country.tolist(), to='ISO3', not_found='NA'))
survey2018['Country'] = converted_country

In [5]:
''' 
Hobby : no NA field 
Country : converted to iso3, other countries is mapped to NA
Student : {  No -> No, Full time -> FT, part time -> PT, NA -> No  }
Years of coding : just keep as it is --> as there are multiple reason why it is NA (coding as hobby / student)
Satisfaction : Nan is map to mean of the data
'''
# display(survey2018['JobSatisfaction'].head(20))
# display(survey2018[survey2018.JobSatisfaction == 'No']) 


''' cleaning student status '''
def mapStudentStatus(x):
    return 'PT' if x == 'Yes, part-time' else 'FT' if x == 'Yes, full-time' else 'No' 
survey2018['Student'] = survey2018['Student'].map(mapStudentStatus, na_action='ignore')
survey2018['Student'].fillna(value='No', inplace=True)

''' cleaning year of satisfaction (can only run once) '''
SATISFACTION_MAP = {'Extremely satisfied': 7, 'Moderately satisfied': 6, 'Slightly satisfied': 5, 
                    'Neither satisfied nor dissatisfied': 4, 'Slightly dissatisfied': 3, 'Moderately dissatisfied': 2,
                    'Extremely dissatisfied': 1}
survey2018['JobSatisfaction'] = survey2018['JobSatisfaction'].map(lambda x: SATISFACTION_MAP[x], na_action='ignore')
survey2018['JobSatisfaction'].fillna(survey2018.JobSatisfaction.mean(), inplace=True)
survey2018['CareerSatisfaction'] = survey2018['CareerSatisfaction'].map(lambda x: SATISFACTION_MAP[x], na_action='ignore')
survey2018['CareerSatisfaction'].fillna(survey2018.CareerSatisfaction.mean(), inplace=True)

display(survey2018.head(5), survey2018.tail(5))

# survey2018[(survey2018.Student.isna()) | (survey2018.JobSatisfaction.isna()) | (survey2018.CareerSatisfaction.isna())]

,Hobby,Country,Student,YearsCoding,YearsCodingProf,JobSatisfaction,CareerSatisfaction,ConvertedSalary,LanguageWorkedWith,LanguageDesireNextYear,...,StackOverflowVisit,StackOverflowParticipate,WakeTime,HoursComputer,HoursOutside,SkipMeals,Exercise,Gender,RaceEthnicity,Age
0,Yes,KEN,No,3-5 years,3-5 years,7.0,7.0,NaN,JavaScript;Python;HTML;CSS,JavaScript;Python;HTML;CSS,...,Multiple times per day,I have never participated in Q&A on Stack Over...,Between 5:00 - 6:00 AM,9 - 12 hours,1 - 2 hours,Never,3 - 4 times per week,Male,Black or of African descent,25 - 34 years old
1,Yes,GBR,No,30 or more years,18-20 years,2.0,4.0,70841.0,JavaScript;Python;Bash/Shell,Go;Python,...,A few times per month or weekly,A few times per month or weekly,Between 6:01 - 7:00 AM,5 - 8 hours,30 - 59 minutes,Never,Daily or almost every day,Male,White or of European descent,35 - 44 years old
2,Yes,USA,No,24-26 years,6-8 years,6.0,6.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,No,USA,No,18-20 years,12-14 years,4.0,3.0,NaN,C#;JavaScript;SQL;TypeScript;HTML;CSS;Bash/Shell,C#;JavaScript;SQL;TypeScript;HTML;CSS;Bash/Shell,...,A few times per week,A few times per month or weekly,Between 6:01 - 7:00 AM,9 - 12 hours,Less than 30 minutes,3 - 4 times per week,I don't typically exercise,Male,White or of European descent,35 - 44 years old
4,Yes,ZAF,PT,6-8 years,0-2 years,5.0,6.0,21426.0,C;C++;Java;Matlab;R;SQL;Bash/Shell,Assembly;C;C++;Matlab;SQL;Bash/Shell,...,Daily or almost daily,Less than once per month or monthly,Before 5:00 AM,Over 12 hours,1 - 2 hours,Never,3 - 4 times per week,Male,White or of European descent,18 - 24 years old


,Hobby,Country,Student,YearsCoding,YearsCodingProf,JobSatisfaction,CareerSatisfaction,ConvertedSalary,LanguageWorkedWith,LanguageDesireNextYear,...,StackOverflowVisit,StackOverflowParticipate,WakeTime,HoursComputer,HoursOutside,SkipMeals,Exercise,Gender,RaceEthnicity,Age
91201,Yes,USA,No,3-5 years,NaN,5.042078,5.141915,NaN,NaN,NaN,...,Less than once per month or monthly,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91202,Yes,KEN,No,3-5 years,NaN,5.042078,5.141915,NaN,NaN,NaN,...,Daily or almost daily,Daily or almost daily,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91203,Yes,BRA,No,21-23 years,NaN,5.042078,5.141915,NaN,NaN,NaN,...,Multiple times per day,A few times per month or weekly,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91204,Yes,GIN,No,NaN,NaN,5.042078,5.141915,NaN,NaN,NaN,...,I have never visited Stack Overflow (before to...,NaN,Before 5:00 AM,NaN,NaN,Never,I don't typically exercise,NaN,NaN,NaN
91205,Yes,NPL,No,0-2 years,0-2 years,5.042078,5.141915,NaN,NaN,NaN,...,Multiple times per day,A few times per week,I do not have a set schedule,5 - 8 hours,1 - 2 hours,1 - 2 times per week,I don't typically exercise,NaN,NaN,NaN


,Hobby,Country,Student,YearsCoding,YearsCodingProf,JobSatisfaction,CareerSatisfaction,ConvertedSalary,LanguageWorkedWith,LanguageDesireNextYear,...,StackOverflowVisit,StackOverflowParticipate,WakeTime,HoursComputer,HoursOutside,SkipMeals,Exercise,Gender,RaceEthnicity,Age


In [6]:
'''
Age : NaN -> 'NA'
Gender : NaN -> 'NA'
'''
# survey2018[survey2018['Age'].isna()]     # 26632 rows does not provide age
survey2018.Age.fillna(value='NA', inplace=True)
# survey2018[survey2018['Gender'].isna()]    # 26738 rows does not provide gender
survey2018.Gender.fillna(value='NA', inplace=True)

In [7]:
''' 
Converted Salary : fill na with student -> the converted salary = 0 , else leave as NA --> TBC
'''
# TODO
survey2018.loc[(survey2018.ConvertedSalary.isna()) & (survey2018.Student != 'No'), 'ConvertedSalary'] = 0
# survey2018[(survey2018.ConvertedSalary.isna())]  # 29716 rows does not provide salary

In [8]:
'''
LanguageWorkedWith to FrameworkDesireNextYear : replace na with empty list, other split into list of lang / framework
'''
# clean all the fields (can only run once)

# survey2018[survey2018.LanguageWorkedWith.isna()]              # 13021 rows does not provide prog languages
# survey2018[survey2018.LanguageDesireNextYear.isna()]          # 18000 rows does not provide prog languages
# survey2018[survey2018.DatabaseWorkedWith.isna()]              # 24941 rows does not provide db 
# survey2018[survey2018.DatabaseDesireNextYear.isna()]          # 33709 rows does not provide db 
# survey2018[survey2018.PlatformWorkedWith.isna()].shape        # 25210 rows does not provide platform 
# survey2018[survey2018.PlatformDesireNextYear.isna()].shape    # 29857 rows does not provide platform 
# survey2018[survey2018.FrameworkWorkedWith.isna()].shape       # 39587 rows does not provide framework 
# survey2018[survey2018.FrameworkDesireNextYear.isna()].shape   # 35990 rows does not provide framework 


''' apply function to clean these fields '''
survey2018['LanguageWorkedWith'] = survey2018.LanguageWorkedWith.apply(lambda x: [] if pd.isna(x) else x.split(';'))
survey2018['LanguageDesireNextYear'] = survey2018.LanguageDesireNextYear.apply(lambda x: [] if pd.isna(x) else x.split(';'))
survey2018['DatabaseWorkedWith'] = survey2018.DatabaseWorkedWith.apply(lambda x: [] if pd.isna(x) else x.split(';'))
survey2018['DatabaseDesireNextYear'] = survey2018.DatabaseDesireNextYear.apply(lambda x: [] if pd.isna(x) else x.split(';'))
survey2018['PlatformWorkedWith'] = survey2018.PlatformWorkedWith.apply(lambda x: [] if pd.isna(x) else x.split(';'))
survey2018['PlatformDesireNextYear'] = survey2018.PlatformDesireNextYear.apply(lambda x: [] if pd.isna(x) else x.split(';'))
survey2018['FrameworkWorkedWith'] = survey2018.FrameworkWorkedWith.apply(lambda x: [] if pd.isna(x) else x.split(';'))
survey2018['FrameworkDesireNextYear'] = survey2018.FrameworkDesireNextYear.apply(lambda x: [] if pd.isna(x) else x.split(';'))


# survey2018[survey2018.LanguageWorkedWith.isna()].shape
# survey2018[survey2018.LanguageDesireNextYear.isna()].shape    
# survey2018[survey2018.DatabaseWorkedWith.isna()].shape        
# survey2018[survey2018.DatabaseDesireNextYear.isna()].shape   
# survey2018[survey2018.PlatformWorkedWith.isna()].shape         
# survey2018[survey2018.PlatformDesireNextYear.isna()].shape    
# survey2018[survey2018.FrameworkWorkedWith.isna()].shape        
# survey2018[survey2018.FrameworkDesireNextYear.isna()].shape    


In [17]:
''' mid progress check (only year of coding x2 and converted salary has NA values)'''

# survey2018[survey2018.Student.isnull()].shape
# survey2018.loc[:, 'LanguageWorkedWith':'FrameworkDesireNextYear'].isnull().values.any()

False

In [18]:
survey2018.to_pickle('./survey2018_half_cleaned.pkl')

In [ ]:
''' Other fields left for later.... '''

# *************** Separator for old code ***************


In [4]:
# ''' cleaning function '''
# # remove NA in the questions that ask the user to rank different choices
# choices_ques = [col for col in survey_result_pub2018 if col[-1].isdigit()]
# priority_Arranging_Ques = [col for col in choices_ques if not any([k in col for k in ['Tools', 'Agree']])]

# def remove_NA_priorityQ(item):
#     return 0 if item == 'NA' else int(item)

# for col in priority_Arranging_Ques:
#     survey_result_pub2018[col] = survey_result_pub2018[col].map(remove_NA_priorityQ)
# survey_result_pub2018[priority_Arranging_Ques]

,AssessJob1,AssessJob2,AssessJob3,AssessJob4,AssessJob5,AssessJob6,AssessJob7,AssessJob8,AssessJob9,AssessJob10,...,JobEmailPriorities5,JobEmailPriorities6,JobEmailPriorities7,AdsPriorities1,AdsPriorities2,AdsPriorities3,AdsPriorities4,AdsPriorities5,AdsPriorities6,AdsPriorities7
0,10,7,8,1,2,5,3,4,9,6,...,1,4,3,1,5,4,7,2,6,3
1,1,7,10,8,2,5,4,3,6,9,...,2,6,7,3,5,1,4,6,7,2
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,5,7,1,2,6,4,3,10,9,...,1,4,5,2,3,4,6,1,7,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98850,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98851,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98852,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98853,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
# all(survey_result_pub2018['AdsPriorities2'].map(lambda x: isinstance(x, int))) # true after mapping
# # all(survey_result_pub2018[('AdsPriorities3')].map(lambda x: isinstance(x, str)))  # true in original df

True

In [25]:
# survey_result_pub2018.shape

(98855, 129)

In [3]:
# ''' cleaning function '''
# # remove the rows that the country is NA
# survey_result_pub2018 = survey_result_pub2018[survey_result_pub2018.Country != 'NA']

In [30]:
# survey_result_pub2018.shape

(98443, 129)

In [17]:
# survey_result_pub2018.groupby('Country').agg('count')
# ['Respondent' > 100]

TypeError: '>' not supported between instances of 'str' and 'int'

## Looking at the dataset of 2011

In [17]:
''' testing the loading of data sets ''' 
# survey_result_pub2013 = pd.read_csv('part_of_data/2013 Stack Overflow Survey Responses.csv', keep_default_na = False)
# survey_result_pub2014 = pd.read_csv('part_of_data/2014 Stack Overflow Survey Responses.csv', keep_default_na = False)
# survey_result_pub2015 = pd.read_csv('part_of_data/2015 Stack Overflow Developer Survey Responses.csv', keep_default_na = False)
# survey_result_pub2016 = pd.read_csv('part_of_data/2016 Stack Overflow Survey Results/2016 Stack Overflow Survey Responses.csv', keep_default_na = False)
# survey_result_pub2017 = pd.read_csv('part_of_data/developer_survey_2017/survey_results_public.csv', keep_default_na = False)
# survey_result_pub2018 = pd.read_csv('part_of_data/developer_survey_2018/survey_results_public.csv', keep_default_na = False)
# survey_result_pub2019 = pd.read_csv('part_of_data/developer_survey_2019/survey_results_public.csv', keep_default_na = False)
